In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cmap
import matplotlib.colors as mpl_colors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import os
import tensorflow as tf

In [2]:
pip install tensorflow_decision_forests

In [3]:
import tensorflow_decision_forests as tfdf

In [4]:
DATA_DIR = "../input/tabular-playground-series-sep-2022/"
os.listdir(DATA_DIR)

In [5]:
train_df = pd.read_csv(DATA_DIR+'train.csv', index_col=['row_id','date'])
train_df.head()

In [6]:
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [7]:
# Split the dataset into a training and testing dataset.
train_ds_pd, test_ds_pd = split_dataset(train_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

# Name of the label column.
label = "num_sold"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

In [8]:
model_regression_1 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

# Train the model.
model_regression_1.fit(x=train_ds)

In [9]:
# Evaluate the model on the test dataset.
model_regression_1.compile(metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
evaluation = model_regression_1.evaluate(test_ds, return_dict=True)

print(evaluation)

In [10]:
test_df = pd.read_csv(DATA_DIR+'test.csv', index_col=['row_id','date'])

In [11]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, task=tfdf.keras.Task.REGRESSION)

In [12]:
preds = model_regression_1.predict(test_ds)

In [13]:
submission_df = pd.read_csv(DATA_DIR+'sample_submission.csv')
submission_df['num_sold'] = preds
submission_df['num_sold'] = submission_df['num_sold'].astype('int')

In [14]:
submission_df.to_csv('submission.csv', header=True, index=False)